In [ ]:
%load_ext autoreload
%autoreload 2

In [17]:
from functions import main, save_df
from graphics import *

df_all = main("https://www.cheapshark.com/api/1.0/deals")

In [ ]:
df_all.head()

In [ ]:
df_games = df_all[['gameID', 'title', 'release_date', 'scores']]
save_df(df_games, "games")

In [ ]:
df_shops = df_all[['store_id', 'store_name']]
save_df(df_shops, "shops")

In [ ]:
df_gameunits = df_all[['gameID', 'store_id', 'og_price', 'dc_price']]
save_df(df_gameunits, "units")

In [ ]:
df_games_genres = df_all[['gameID', 'genre']]
save_df(df_games_genres, "games_genres")

In [ ]:
df_genres = df_all[['genre']]
save_df(df_genres, "genres")

In [24]:
#average_disc_vs_shop_bar(deals_df)
# min_max_disc_vs_shop_bar(deals_df)
# linear_graph_discounts_vs_punct_linear(deals_df)
# genre_vs_mean_discount_bar(unroll_list_from_dfcolumn(deals_df, 'genre'))
# circular_pie_chart_genres(unroll_list_from_dfcolumn(deals_df, 'genre'))
# linear_graph_discounts_vs_releasedate(clean_df_releases(deals_df))